## Extract trails
* This notebook will concatenate all epochs assoicated with a trail in to one data point.
* The goal of this is to try to describe data as congruent or incongruent.
    * In other words a trail will be described as congruent or incongruent

In [1]:
%run ../preprocessing/Artifact_Removal/preprocecssing_helpers.ipynb
%run ../preprocessing/StimCodes.ipynb
import os

import mne
from mne.preprocessing import ICA
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib
matplotlib.rcParams['figure.figsize'] = (50.0, 30.0)
matplotlib.rcParams.update({'font.size': 15})

from sklearn.preprocessing import StandardScaler, normalize

In [2]:
path = 'E:\EpochedEEG'
epoch_files = os.listdir(path)
file = os.path.join(path,epoch_files[0])
epoch = mne.read_epochs(file, preload = True)

Reading E:\EpochedEEG\20131216_1441_epo.fif ...
    Found the data of interest:
        t =    -199.22 ...     500.00 ms
        0 CTF compensation matrices available
5424 matching events found
Applying baseline correction (mode: mean)
5424 matching events found
Applying baseline correction (mode: mean)
Not setting metadata
0 projection items activated


In [3]:
epoch = update_stim_code_event_ids(epoch)

In [4]:
epoch.event_id

{'trail_start': 31,
 'left_button': 131,
 'right_button': 132,
 'congruent': 133,
 'incongruent': 134,
 'correct': 129,
 'incorrect': 130,
 't_baby': 135,
 't_bell': 136,
 't_bird': 137,
 't_burp': 138,
 't_dog': 139,
 't_drum': 140,
 't_knock': 141,
 't_laugh': 142,
 't_phone': 143,
 't_train': 144,
 't_water': 145,
 'f_baby': 155,
 'f_bell': 156,
 'f_bird': 157,
 'f_burp': 158,
 'f_dog': 159,
 'f_drum': 160,
 'f_knock': 161,
 'f_laugh': 162,
 'f_phone': 163,
 'f_train': 164,
 'f_water': 165,
 'b-f2-wrd': 1,
 'b-f1-wrd': 2,
 'b-tg-wrd': 3,
 'b-f2-spk': 4,
 'b-f1-spk': 5,
 'b-tg-spk': 6,
 'b-f2-pic': 7,
 'b-f1-pic': 8,
 'b-tg-pic': 9,
 'b-f2-snd': 10,
 'b-f1-snd': 11,
 'b-tg-snd': 12,
 'f-f2-wrd': 13,
 'f-f1-wrd': 14,
 'f-tg-wrd': 15,
 'f-f2-spk': 16,
 'f-f1-spk': 17,
 'f-tg-spk': 18,
 'f-f2-pic': 19,
 'f-f1-pic': 20,
 'f-tg-pic': 21,
 'f-f2-snd': 22,
 'f-f1-snd': 23,
 'f-tg-snd': 24}

In [8]:
events = epoch.events[:,-1]
event_ids = epoch.event_id
print("len events:", len(events))

len events: 5424


In [9]:
epoch.get_data().shape

(5424, 129, 180)

In [10]:
#combining events of a trail into 1 list
trials = []
event_trial = []
for event in events:
    if event == 31 and event_trial != []:
        trials.append(event_trial)
        event_trial = []
        event_trial.append(event)
    else:
        event_trial.append(event)
# print(np.array(trails))

In [11]:
trials_and_events = []
trial_dict = {}
for trial in trials:
    for event in trial:
        for key, value in event_ids.items():
            if value == event:
                trial_dict[key] = event
    trials_and_events.append(trial_dict)
    trial_dict = {}
trials_and_events[:5]

[{'left_button': 131},
 {'trail_start': 31,
  't_train': 144,
  'f_laugh': 162,
  'b-f1-pic': 8,
  'b-tg-wrd': 3,
  'b-f2-pic': 7,
  'right_button': 132,
  'incongruent': 134,
  'incorrect': 130},
 {'trail_start': 31,
  't_laugh': 142,
  'f_laugh': 162,
  'b-f1-pic': 8,
  'b-tg-wrd': 3,
  'b-f2-pic': 7,
  'right_button': 132,
  'congruent': 133,
  'correct': 129},
 {'trail_start': 31,
  't_laugh': 142,
  'f_laugh': 162,
  'b-f1-pic': 8,
  'b-tg-wrd': 3,
  'b-f2-pic': 7,
  'right_button': 132,
  'congruent': 133},
 {'trail_start': 31,
  't_laugh': 142,
  'f_laugh': 162,
  'b-f1-pic': 8,
  'b-tg-wrd': 3,
  'b-f2-pic': 7,
  'right_button': 132,
  'congruent': 133,
  'correct': 129}]

In [12]:
#remove dead trail at the beginning of study
del trials_and_events[0]

###  Combine epochs by trail (event_ids are combined, but combining epochs is a different task)

In [13]:
epoch.get_data().shape

(5424, 129, 180)

### combine epochs and give them a label equal to Audio or Visual 

In [12]:
# creating a new dictionary where each trail will equal one of the values in the dictinary Mode_lexi that is defined in 
# ../preprocessing/StimCodes.ipynb
trial_event_ids = []
for trial in trials_and_events:
    #check individual stim-codes and match them with corresponding combination defined in stimcodes.ipynb
    stim_combination = []
    
    for key in trial:
#         print(trial[key])
        if int(trial[key]) >= 1 and int(trial[key]) <= 24:
            stim_combination.append(trial[key])
#             print(stim_combination)
        if len(stim_combination) == 3: # stim-code combination complete, it can now be matched to a value in Mode_lexi
            stim_combination_tuple = (stim_combination[0],stim_combination[1],stim_combination[2])
#             print(stim_combination_tuple)
            
            for stim_combo_key in modal_lexi:
                stim_combo1 = modal_lexi[stim_combo_key][0]
                stim_combo2 = modal_lexi[stim_combo_key][1]

                if stim_combination_tuple == stim_combo1 or stim_combination == stim_combo2:
                    dict = {stim_combo_key : trial}      
                    trial_event_ids.append(dict)
            break;
 

In [15]:
trial_event_ids[15:30] #this trial has stim-code combination VVLN, which is a visual stimuli 

[{'VVLN': {'trail_start': 31,
   't_laugh': 142,
   'f_laugh': 162,
   'b-f1-pic': 8,
   'b-tg-wrd': 3,
   'right_button': 132,
   'b-f2-pic': 7,
   'congruent': 133}},
 {'VVLN': {'trail_start': 31,
   't_train': 144,
   'f_laugh': 162,
   'b-f1-pic': 8,
   'b-tg-wrd': 3,
   'b-f2-pic': 7,
   'left_button': 131,
   'incongruent': 134,
   'correct': 129}},
 {'VVLN': {'trail_start': 31,
   't_laugh': 142,
   'f_laugh': 162,
   'b-f1-pic': 8,
   'b-tg-wrd': 3,
   'b-f2-pic': 7,
   'right_button': 132,
   'congruent': 133}},
 {'VVLN': {'trail_start': 31,
   't_train': 144,
   'f_laugh': 162,
   'b-f1-pic': 8,
   'b-tg-wrd': 3,
   'b-f2-pic': 7,
   'left_button': 131,
   'incongruent': 134}},
 {'VVLN': {'trail_start': 31,
   't_train': 144,
   'f_laugh': 162,
   'b-f1-pic': 8,
   'b-tg-wrd': 3,
   'b-f2-pic': 7,
   'left_button': 131,
   'incongruent': 134}},
 {'VVNN': {'trail_start': 31,
   't_drum': 140,
   'f_drum': 160,
   'b-f1-pic': 8,
   'b-tg-pic': 9,
   'b-f2-pic': 7,
   'right_but

In [16]:
updated_epochs = epochs[1:]#dropped first epoch - it is a dead epoch
trial_epochs = []

for epoch in updated_epochs:
    2
epoch.get_data()[:2,:,:].shape

NameError: name 'epochs' is not defined

In [18]:
#combine epochs into trials with index of epochs of a trial
trials = []
event_trial = []
start_index = 0
current_index = 0

for i in range(len(events)):
    if events[i] == 31 and event_trial != []:
        
        trials.append([(start_index, current_index),event_trial])
        
        event_trial = []
        start_index+=1
        event_trial.append(events[i])
    else:
        current_index+=1
        event_trial.append(events[i])

In [20]:
# now I can create a list of epoch objects indicating trials
epoch_trials = []
for trial in trials:
    start_index, end_index = trial[0]
    epoch_trials.append(epoch.get_data()[start_index:end_index,:,:])
    

In [24]:
np.array(epoch_trials).shape

(1, 129, 180)